In [16]:
import collections
import datetime
import os
import platform
from collections import Counter, OrderedDict
from datetime import date, timedelta

import numpy as np
import pandas as pd
import requests
import sklearn
from dateutil import rrule

output_folder = "F:/Reading/年年月月/The Economist/mp3"

In [53]:
def allsaturdays(year):
    ###
    # This function get all Saturdays in a specified year
    ###
    
    d = date(year, 1, 1)  # January 1st
    t = timedelta((7 + 5 - d.weekday()) % 7)  # First Saturday
    d = d + t
    while d.year == year:
        yield d
        d += timedelta(days=7)

def get_saturdays(date_start: str, date_end:str =None):
    # https://stackoverflow.com/questions/61948459/how-to-get-a-list-of-some-specific-weekdays-within-a-date-range-in-python
    # the input format is "%Y-%m-%d"
    date_start = datetime.datetime.strptime(date_start, "%Y-%m-%d")
    if date_end is None:
        date_end = datetime.datetime.today()
    else:
        date_end = datetime.datetime.strptime(date_end, "%Y-%m-%d")

    result = []
    while date_start <= date_end:
        if date_start.weekday() == 5:   #0 == Monday
            result.append(date_start) 
        date_start += datetime.timedelta(days=1)

    return result
        
def getIssueNo(issue_date):

    origin_date = datetime.datetime(2021, 4, 10)
    origin_issue_no = 9240

    if issue_date >= origin_date:
        start_date = origin_date
        end_date = issue_date
        sign = 1
    else:
        start_date = issue_date
        end_date = origin_date
        sign = -1
    # this will return a generator of date
    # starting from start_date
    weeks = rrule.rrule(rrule.WEEKLY, dtstart=start_date, until=end_date)
    delta = 0
    for week in weeks:
        # skip Chistmas Saturday
        if week.month == 12 and week.day == 25:
            continue
        else:
            delta += 1
    delta = sign * (delta - 1)

    issue_no = origin_issue_no + delta

    return issue_no
    
def downloadFile(file_url, output_fname, output_path):
    local_fpath = os.path.join(output_path, output_fname)
    curr_system = platform.system()
    if curr_system == "Windows":
        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with open(local_fpath, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunck)
        return f"{local_fpath} Done!"
    else:
        import wget
        wget.download(file_url, out=local_fpath)
        return f"{local_fpath} Done!"

def getEcoAudio(year, month, day, output_path="./", show_url_only=True):

    issue_date = datetime.datetime(year, month, day)
    issue_no = getIssueNo(issue_date)
    issue_date_formatted = issue_date.strftime("%Y%m%d")
    
    file_name = "Issue_{issue}_{date}_The_Economist_Full_edition.zip".format(
        date=issue_date_formatted, issue=issue_no
    )
    download_url = "http://audiocdn.economist.com/sites/default/files/AudioArchive/{year}/{date}/{file_name}".format(
        year=str(year), date=issue_date_formatted, file_name=file_name
    )

    print(download_url)
    if show_url_only:
        return

    downloadFile(
        download_url,
        output_fname=f"TheEconomist_AudioEdition_{issue_date_formatted}.zip",
        output_path=output_path,
    )

In [ ]:
for issue_date in get_saturdays('2022-01-01'):
    try:
        getEcoAudio(
            issue_date.year,
            issue_date.month,
            issue_date.day,
            output_path=output_folder,
            show_url_only=True,
        )
    except Exception as e:
        print(issue_date)
        print(e)
        continue

In [55]:
for issue_date in allsaturdays(2021):
    try:
        getEcoAudio(
            issue_date.year,
            issue_date.month,
            issue_date.day,
            output_path=output_folder,
            show_url_only=True,
        )
    except Exception as e:
        print(issue_date)
        print(e)
        continue

http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210102/Issue_9226_20210102_The_Economist_Full_edition.zip
http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210109/Issue_9227_20210109_The_Economist_Full_edition.zip
http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210116/Issue_9228_20210116_The_Economist_Full_edition.zip
http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210123/Issue_9229_20210123_The_Economist_Full_edition.zip
http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210130/Issue_9230_20210130_The_Economist_Full_edition.zip
http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210206/Issue_9231_20210206_The_Economist_Full_edition.zip
http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210213/Issue_9232_20210213_The_Economist_Full_edition.zip
http://audiocdn.economist.com/sites/default/files/AudioArchive/2021/20210220/Issue_9233_20210220_The_Eco